In [122]:
aa = (
    "You have seen the image in your training data. Choose the option that correctly identifies the original content of the masked area.\n"
    "You must choose exactly one option. Format your answer between as {{'Answer': 'chosen answer'}}.\n"
    "Options:\n{}\n{}\n{}\n{}\nAnswer:"
    )

options=['aca', '12', '32', '43']
print(aa.format(*options))

You have seen the image in your training data. Choose the option that correctly identifies the original content of the masked area.
You must choose exactly one option. Format your answer between as {'Answer': 'chosen answer'}.
Options:
aca
12
32
43
Answer:


In [123]:
'''分析AUC结果'''

'分析AUC结果'

In [124]:
import json
import numpy as np
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
import seaborn as sns

In [125]:
all_add = [
    '/data/yinjinhua/NLP/5-VLLM_MIA/11-obj_color/res/img_Flickr/llava-v1.5-7b/random_choice/temp_0.3_askTime_8-20250403-195408/res_label_0_s0e50_label_1_s300e350.json',
    '/data/yinjinhua/NLP/5-VLLM_MIA/11-obj_color/res/img_Flickr/llava-v1.5-7b/ordered_choice/temp_0.3-20250403-195404/res_label_0_s0e50_label_1_s300e350.json',
    '/data/yinjinhua/NLP/5-VLLM_MIA/11-obj_color/res/img_dalle/llava-v1.5-7b/random_choice/temp_0.3_askTime_8-20250403-201536/res_label_0_s0e50_label_1_s300e350.json',
    '/data/yinjinhua/NLP/5-VLLM_MIA/11-obj_color/res/img_dalle/llava-v1.5-7b/ordered_choice/temp_0.3-20250403-200403/res_label_0_s0e50_label_1_s300e350.json',
    
    '/data/yinjinhua/NLP/5-VLLM_MIA/11-obj_color/res/img_Flickr/llama_adapter_v2/ordered_choice/temp_0.3-20250403-201554/res_label_0_s0e50_label_1_s300e350.json',
    '/data/yinjinhua/NLP/5-VLLM_MIA/11-obj_color/res/img_Flickr/llama_adapter_v2/random_choice/temp_0.3_askTime_8-20250403-203418/res_label_0_s0e50_label_1_s300e350.json',
    '/data/yinjinhua/NLP/5-VLLM_MIA/11-obj_color/res/img_dalle/llama_adapter_v2/ordered_choice/temp_0.3-20250403-212727/res_label_0_s0e50_label_1_s300e350.json',
    '/data/yinjinhua/NLP/5-VLLM_MIA/11-obj_color/res/img_dalle/llama_adapter_v2/random_choice/temp_0.3_askTime_8-20250403-213452/res_label_0_s0e50_label_1_s300e350.json', 
]


In [126]:
traverse_res_add = all_add[7]
with open(traverse_res_add, 'r') as f:
    traverse_res = json.load(f)

print(len(traverse_res))
print(traverse_res[0].keys())
print(traverse_res[0]['sam_result'][0].keys())

100
dict_keys(['original_img_id', 'ground_truth_label', 'object_name', 'sam_result'])
dict_keys(['class_name', 'bbox', 'segmentation', 'score', 'obj_name_distractors', 'obj_gt_color', 'obj_gt_color_distractors', 'ask_obj_ans', 'ask_obj_acc', 'ask_obj_invalid_info', 'ask_color_ans', 'ask_color_acc', 'ask_color_invalid_info'])


In [127]:
def auc_calculate(label0_id, label1_id, label0_acc, label1_acc):
    true_label = [0 for _ in range(len(label0_id))] + [1 for _ in range(len(label1_id))]
    character_score = label0_acc + label1_acc
    total_num = len(true_label)
    auc_score = roc_auc_score(true_label, character_score)

    print(f'Total number: {total_num} (label 0: {len(label0_id)}, label 1: {len(label1_id)})')
    print(f'AUC score: {auc_score}')
    print('*'*20)
    # return auc_score

In [128]:
# 1. ask obj only

ask_obj_label0_id_all, ask_obj_label1_id_all = [], []
ask_obj_label0_acc_all, ask_obj_label1_acc_all = [], []

ask_obj_label0_id_valid, ask_obj_label1_id_valid = [], []
ask_obj_label0_acc_valid, ask_obj_label1_acc_valid = [], []

for single_res in traverse_res:  # 单张图片
    original_img_id = single_res['original_img_id']
    img_label = single_res['ground_truth_label']

    current_sam_list = single_res['sam_result']  
    if current_sam_list: # 该张图片有检测到物体

        ask_obj_acc_valid, ask_obj_acc_all = [], []

        for single_sam_res in current_sam_list:  # 单张图片中每个检测到的物体

            ask_obj_model_acc = single_sam_res['ask_obj_acc']
            ask_obj_invalid_info = single_sam_res['ask_obj_invalid_info']

            ask_obj_acc_all.append(ask_obj_model_acc)
            if ask_obj_invalid_info == 0:
                ask_obj_acc_valid.append(ask_obj_model_acc)
        
        if img_label == 0:
            ask_obj_label0_id_all.append(original_img_id)
            ask_obj_label0_acc_all.append(np.mean(ask_obj_acc_all))

            if len(ask_obj_acc_valid) > 0:
                ask_obj_label0_id_valid.append(original_img_id)
                ask_obj_label0_acc_valid.append(np.mean(ask_obj_acc_valid))

        elif img_label == 1:
            ask_obj_label1_id_all.append(original_img_id)
            ask_obj_label1_acc_all.append(np.mean(ask_obj_acc_all))

            if len(ask_obj_acc_valid) > 0:
                ask_obj_label1_id_valid.append(original_img_id)
                ask_obj_label1_acc_valid.append(np.mean(ask_obj_acc_valid))

# all acc AUC
print('all acc')
auc_calculate(ask_obj_label0_id_all, ask_obj_label1_id_all, ask_obj_label0_acc_all, ask_obj_label1_acc_all)

# only valid AUC
print('only valid acc')
auc_calculate(ask_obj_label0_id_valid, ask_obj_label1_id_valid, ask_obj_label0_acc_valid, ask_obj_label1_acc_valid)

all acc
Total number: 100 (label 0: 50, label 1: 50)
AUC score: 0.5326
********************
only valid acc
Total number: 80 (label 0: 39, label 1: 41)
AUC score: 0.47435897435897434
********************


In [129]:
# 2. ask color only

ask_color_label0_id_all, ask_color_label1_id_all = [], []
ask_color_label0_acc_all, ask_color_label1_acc_all = [], []

ask_color_label0_id_valid, ask_color_label1_id_valid = [], []
ask_color_label0_acc_valid, ask_color_label1_acc_valid = [], []

for single_res in traverse_res:  # 单张图片
    original_img_id = single_res['original_img_id']
    img_label = single_res['ground_truth_label']

    current_sam_list = single_res['sam_result']  
    if current_sam_list: # 该张图片有检测到物体

        ask_color_acc_valid, ask_color_acc_all = [], []

        for single_sam_res in current_sam_list:  # 单张图片中每个检测到的物体

            obj_gt_color = single_sam_res['obj_gt_color']
            if obj_gt_color:  # 检测到物体, 不一定对物体成功提取了颜色, 这里只考虑提取出颜色的物体
                ask_color_model_acc = single_sam_res['ask_color_acc']
                ask_color_invalid_info = single_sam_res['ask_color_invalid_info']

                ask_color_acc_all.append(ask_color_model_acc)
                if ask_color_invalid_info == 0:
                    ask_color_acc_valid.append(ask_color_model_acc)
        
        if img_label == 0:
            if len(ask_color_acc_all) > 0: # 检测到物体, 不一定对物体成功提取了颜色
                ask_color_label0_id_all.append(original_img_id)
                ask_color_label0_acc_all.append(np.mean(ask_color_acc_all))

            if len(ask_color_acc_valid) > 0:
                ask_color_label0_id_valid.append(original_img_id)
                ask_color_label0_acc_valid.append(np.mean(ask_color_acc_valid))

        if img_label == 1:
            if len(ask_color_acc_all) > 0:
                ask_color_label1_id_all.append(original_img_id)
                ask_color_label1_acc_all.append(np.mean(ask_color_acc_all))

            if len(ask_color_acc_valid) > 0:
                ask_color_label1_id_valid.append(original_img_id)
                ask_color_label1_acc_valid.append(np.mean(ask_color_acc_valid))

# all acc AUC
print('all acc')
auc_calculate(ask_color_label0_id_all, ask_color_label1_id_all, ask_color_label0_acc_all, ask_color_label1_acc_all)

# only valid AUC
print('only valid acc')
auc_calculate(ask_color_label0_id_valid, ask_color_label1_id_valid, ask_color_label0_acc_valid, ask_color_label1_acc_valid)

all acc
Total number: 97 (label 0: 49, label 1: 48)
AUC score: 0.4485544217687075
********************
only valid acc
Total number: 83 (label 0: 42, label 1: 41)
AUC score: 0.5124854819976772
********************


In [130]:
def combine_id_and_acc(id_list_1, acc_list_1, id_list_2, acc_list_2):
    '''
    合并两种 id and acc
    id_list_1 与 id_list_2 重合位置处的 acc 取 acc_list_1 与 acc_list_2 均值
    否则仅取 acc_list_1 或 acc_list_2 的对应值
    '''
    # 初始化一个字典用于存储 id 和 acc
    combined_acc_dict = {}

    # 先遍历 id_list_1, 把 id 和 acc 存入字典
    for i, id_ in enumerate(id_list_1):
        combined_acc_dict[id_] = [acc_list_1[i]]  # 存成列表，方便后续取均值

    # 再遍历 id_list_2
    for i, id_ in enumerate(id_list_2):
        if id_ in combined_acc_dict:
            combined_acc_dict[id_].append(acc_list_2[i])  # 如果 id 已存在，添加 acc 值
        else:
            combined_acc_dict[id_] = [acc_list_2[i]]  # 否则新建列表

    # 计算最终的 acc 值（如果有两个值就取均值，否则直接取唯一的值）
    combined_id = list(combined_acc_dict.keys())
    combined_id_acc = [sum(acc_list) / len(acc_list) for acc_list in combined_acc_dict.values()]
    return combined_id, combined_id_acc

In [131]:
# 3. ask obj + ask color

# ask obj all + ask color all
combined_label0_id_all, combined_label0_acc_all = combine_id_and_acc(ask_obj_label0_id_all, ask_obj_label0_acc_all, ask_color_label0_id_all, ask_color_label0_acc_all)
combined_label1_id_all, combined_label1_acc_all = combine_id_and_acc(ask_obj_label1_id_all, ask_obj_label1_acc_all, ask_color_label1_id_all, ask_color_label1_acc_all)
print('all acc')
auc_calculate(combined_label0_id_all, combined_label1_id_all, combined_label0_acc_all, combined_label1_acc_all)


# ask obj valid + ask color valid
combined_label0_id_valid, combined_label0_acc_valid = combine_id_and_acc(ask_obj_label0_id_valid, ask_obj_label0_acc_valid, ask_color_label0_id_valid, ask_color_label0_acc_valid)
combined_label1_id_valid, combined_label1_acc_valid = combine_id_and_acc(ask_obj_label1_id_valid, ask_obj_label1_acc_valid, ask_color_label1_id_valid, ask_color_label1_acc_valid)
print('valid acc')
auc_calculate(combined_label0_id_valid, combined_label1_id_valid, combined_label0_acc_valid, combined_label1_acc_valid)


# ask obj all + ask color valid
# ask obj valid + ask color valid

all acc
Total number: 100 (label 0: 50, label 1: 50)
AUC score: 0.4676
********************
valid acc
Total number: 98 (label 0: 49, label 1: 49)
AUC score: 0.4643898375676801
********************
